In [5]:

### I ran this on a google colab, if running on a local machine, you might need to install different versions.

!pip install relbench[full]
!pip install pyg-lib -f https://data.pyg.org/whl/torch-2.4.0+cu121.html # PyG for working with graphs
!pip install git+https://github.com/pyg-team/pytorch_geometric.git # more PyG
!pip install pytorch_frame[full] #PyTorch Frame for working with tabular data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 66.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.4 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 6.5 MB/s eta 0:00:00a 0:00:010m
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-mezcn3l5
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-mezcn3l5
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 7f844d72d307ab9a105ba6494824cb0205ccbeac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing meta

In [6]:
import os
import time

import torch
import numpy as np
import pandas as pd
import pooch
import pyarrow as pa
import pyarrow.json
from relbench.base import Database, Dataset, Table
from relbench.datasets import get_dataset, get_dataset_names, register_dataset
from relbench.utils import unzip_processor
from sklearn.impute import SimpleImputer

In [7]:
import duckdb
import pandas as pd

from relbench.base import Database, EntityTask, Table, TaskType
from relbench.datasets import get_dataset
from relbench.metrics import accuracy, average_precision, f1, roc_auc
from relbench.tasks import get_task, get_task_names, register_task

In [9]:
### dataset available at https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

class EcommerceDataBase(Dataset):

    val_timestamp = pd.Timestamp(year=2018, month=4, day=1)
    test_timestamp = pd.Timestamp(year=2018, month=8, day=1)

    def make_db(self) -> Database:

        tables = {}
    
        customers = pd.read_csv('olist_customers_dataset.csv').drop(columns=['customer_unique_id'])
        
        orderItems = pd.read_csv('olist_order_items_dataset.csv')
        orderItems['order_item_id'] = orderItems.order_id.astype(str) + '-' + orderItems.order_item_id.astype(str)
        orderItems["shipping_limit_date"] = pd.to_datetime(orderItems["shipping_limit_date"])
        
        payments = pd.read_csv('olist_order_payments_dataset.csv').reset_index().rename(columns={'index':'payment_id'})
        
    
        
        payments['index'] = 'payment_id'
        
        reviews = pd.read_csv('olist_order_reviews_dataset.csv').drop_duplicates(subset=['review_id']).fillna('')
        reviews["timestamp"] = pd.to_datetime(reviews["review_creation_date"])

        
        orders = pd.read_csv('olist_orders_dataset.csv').dropna()
        orders["timestamp"] = pd.to_datetime(orders["order_approved_at"])

        
        products = pd.read_csv('olist_products_dataset.csv')
        product_id = products['product_id']
        products['product_category_name'] = products['product_category_name'].fillna('')
        products_subset = products.drop(columns=['product_category_name', 'product_id'])
        products_imp = SimpleImputer().fit_transform(products_subset)
        products_imp = pd.DataFrame(products_imp, columns=products_subset.columns)
        products_imp['product_id'] = product_id
        products_imp['product_category_name'] = products['product_category_name']
        


        print(products_imp.isna().sum(axis=0))
        print(orders.isna().sum(axis=0))
        print(reviews.isna().sum(axis=0))
        print(payments.isna().sum(axis=0))
        print(orderItems.isna().sum(axis=0))
        print(customers.isna().sum(axis=0))

        
        
        sellers = pd.read_csv('olist_sellers_dataset.csv')
    
        tables['products'] = Table(
            df=pd.DataFrame(products_imp),
            pkey_col='product_id',
            fkey_col_to_pkey_table={},
            time_col=None
        )
        
        tables['customers'] = Table(
            df=pd.DataFrame(customers),
            pkey_col='customer_id',
            fkey_col_to_pkey_table={},
            time_col=None
        )
        
        tables['orderItems'] = Table(
            df=pd.DataFrame(orderItems),
            pkey_col='order_item_id',
            fkey_col_to_pkey_table={
                'order_id': 'orders',
                'product_id': 'products',
                'seller_id': 'sellers'
            },
            time_col=None
        )
        
        tables['payments'] = Table(
            df=pd.DataFrame(payments),
            pkey_col='payment_id',
            fkey_col_to_pkey_table={
                'order_id': 'orders'
            },
            time_col=None
        )
        
        tables['reviews'] = Table(
            df=pd.DataFrame(reviews),
            pkey_col='review_id',
            fkey_col_to_pkey_table={
                'order_id': 'orders'
            },
            time_col='timestamp'
        )
        
        tables['sellers'] = Table(
            df=pd.DataFrame(sellers),
            pkey_col='seller_id',
            fkey_col_to_pkey_table={}
        )
        
        tables['orders'] = Table(
            df=pd.DataFrame(orders),
            pkey_col='order_id',
            fkey_col_to_pkey_table={
                'customer_id': 'customers'
            },
            time_col='timestamp'
        )
        return Database(tables)

In [10]:
class SellerRevenueTask(EntityTask):


    task_type = TaskType.BINARY_CLASSIFICATION
    entity_col = "seller_id"
    entity_table = "sellers"
    time_col = "timestamp"
    target_col = "revenue"
    timedelta = pd.Timedelta(days=30)
    metrics = [average_precision, accuracy, f1, roc_auc]
    num_eval_timestamps = 40

    def make_table(self, db: Database, timestamps: "pd.Series[pd.Timestamp]") -> Table:

        timestamp_df = pd.DataFrame({"timestamp": timestamps})

        orderItems = db.table_dict["orderItems"].df

        df = duckdb.sql(f"""
            select 
                timestamp,
                seller_id,
                sum(price) as revenue
            from 
                timestamp_df t
            left join 
                orderItems oi
            on
                oi.shipping_limit_date <= t.timestamp + INTERVAL '{self.timedelta}'
                and oi.shipping_limit_date > t.timestamp
            group by timestamp, seller_id
        """).df().dropna()

        return Table(
            df=df,
            fkey_col_to_pkey_table={self.entity_col: self.entity_table},
            pkey_col=None,
            time_col=self.time_col,
        )

In [11]:
ecomm_ds = EcommerceDataBase()
db = ecomm_ds.get_db()

seller_revenue_task = SellerRevenueTask(ecomm_ds)

Making Database object from scratch...
(You can also use `get_dataset(..., download=True)` for datasets prepared by the RelBench team.)
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
product_id                    0
product_category_name         0
dtype: int64
order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
timestamp                        0
dtype: int64
review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       0
review_answer_timestamp    0
timestamp                  0
dty

In [18]:
train_table.df.isna().sum(axis=0)

timestamp    0
seller_id    0
revenue      0
dtype: int64

In [ ]:
### from here it's just setting up the model and training it, most of the code is from the relbench documentation
### https://github.com/snap-stanford/relbench/blob/main/tutorials/train_model.ipynb

In [12]:
import numpy as np

from torch.nn import BCEWithLogitsLoss, L1Loss
from relbench.datasets import get_dataset
from relbench.tasks import get_task

train_table = seller_revenue_task.get_table("train")
val_table = seller_revenue_task.get_table("val")
test_table = seller_revenue_task.get_table("test")

out_channels = 1
loss_fn = L1Loss()
tune_metric = "mae"
higher_is_better = False

Making task table for train split from scratch...
(You can also use `get_task(..., download=True)` for tasks prepared by the RelBench team.)
Making Database object from scratch...
(You can also use `get_dataset(..., download=True)` for datasets prepared by the RelBench team.)
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
product_id                    0
product_category_name         0
dtype: int64
order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
timestamp                        0
dtype: int64
review_id                  0
order_id                   0
review_score               0
review_

In [13]:
import os
import math
import numpy as np
from tqdm import tqdm

import torch
import torch_geometric
import torch_frame

# Some book keeping
from torch_geometric.seed import seed_everything

seed_everything(42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)  # check that it's cuda if you want it to run in reasonable time!
root_dir = "./data"

/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/libpyg.so: undefined symbol: _ZNK5torch8autograd4Node4nameEv
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "


cuda


In [14]:
from relbench.modeling.utils import get_stype_proposal

col_to_stype_dict = get_stype_proposal(db)

In [43]:
!pip install -U sentence-transformers # we need another package for text encoding
from typing import List, Optional
from sentence_transformers import SentenceTransformer
from torch import Tensor


class GloveTextEmbedding:
    def __init__(self, device: Optional[torch.device
                                       ] = None):
        self.model = SentenceTransformer(
            "sentence-transformers/average_word_embeddings_glove.6B.300d",
            device=device,
        )

    def __call__(self, sentences: List[str]) -> Tensor:
        return torch.from_numpy(self.model.encode(sentences))

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [73]:
from torch_frame.config.text_embedder import TextEmbedderConfig
from relbench.modeling.graph import make_pkey_fkey_graph

text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=device), batch_size=256
)

data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,  # speficied column types
    text_embedder_cfg=text_embedder_cfg,  # our chosen text encoder
    cache_dir=os.path.join(
        root_dir, f"rel-ecomm_materialized_cache"
    ),  # store materialized graph for convenience
)

/opt/conda/lib/python3.10/site-packages/torch_frame/utils/io.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tf_dict, col_stats = torch.load(path)
/opt/conda/lib/python

In [75]:
from relbench.modeling.graph import get_node_train_table_input, make_pkey_fkey_graph
from torch_geometric.loader import NeighborLoader

loader_dict = {}

for split, table in [
    ("train", train_table),
    ("val", val_table),
    ("test", test_table),
]:
    table_input = get_node_train_table_input(
        table=table,
        task=seller_revenue_task,
    )
    entity_table = table_input.nodes[0]
    loader_dict[split] = NeighborLoader(
        data,
        num_neighbors=[
            128 for i in range(2)
        ],  # we sample subgraphs of depth 2, 128 neighbors per node.
        time_attr="time",
        input_nodes=table_input.nodes,
        input_time=table_input.time,
        transform=table_input.transform,
        batch_size=512,
        temporal_strategy="uniform",
        shuffle=split == "train",
        num_workers=0,
        persistent_workers=False,
    )

/opt/conda/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
/opt/conda/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
/opt/conda/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


In [76]:
from torch.nn import BCEWithLogitsLoss
import copy
from typing import Any, Dict, List

import torch
from torch import Tensor
from torch.nn import Embedding, ModuleDict
from torch_frame.data.stats import StatType
from torch_geometric.data import HeteroData
from torch_geometric.nn import MLP
from torch_geometric.typing import NodeType

from relbench.modeling.nn import HeteroEncoder, HeteroGraphSAGE, HeteroTemporalEncoder


class Model(torch.nn.Module):

    def __init__(
        self,
        data: HeteroData,
        col_stats_dict: Dict[str, Dict[str, Dict[StatType, Any]]],
        num_layers: int,
        channels: int,
        out_channels: int,
        aggr: str,
        norm: str,
        # List of node types to add shallow embeddings to input
        shallow_list: List[NodeType] = [],
        # ID awareness
        id_awareness: bool = False,
    ):
        super().__init__()

        self.encoder = HeteroEncoder(
            channels=channels,
            node_to_col_names_dict={
                node_type: data[node_type].tf.col_names_dict
                for node_type in data.node_types
            },
            node_to_col_stats=col_stats_dict,
        )
        self.temporal_encoder = HeteroTemporalEncoder(
            node_types=[
                node_type for node_type in data.node_types if "time" in data[node_type]
            ],
            channels=channels,
        )
        self.gnn = HeteroGraphSAGE(
            node_types=data.node_types,
            edge_types=data.edge_types,
            channels=channels,
            aggr=aggr,
            num_layers=num_layers,
        )
        self.head = MLP(
            channels,
            out_channels=out_channels,
            norm=norm,
            num_layers=1,
        )
        self.embedding_dict = ModuleDict(
            {
                node: Embedding(data.num_nodes_dict[node], channels)
                for node in shallow_list
            }
        )

        self.id_awareness_emb = None
        if id_awareness:
            self.id_awareness_emb = torch.nn.Embedding(1, channels)
        self.reset_parameters()

    def reset_parameters(self):
        self.encoder.reset_parameters()
        self.temporal_encoder.reset_parameters()
        self.gnn.reset_parameters()
        self.head.reset_parameters()
        for embedding in self.embedding_dict.values():
            torch.nn.init.normal_(embedding.weight, std=0.1)
        if self.id_awareness_emb is not None:
            self.id_awareness_emb.reset_parameters()

    def forward(
        self,
        batch: HeteroData,
        entity_table: NodeType,
    ) -> Tensor:
        seed_time = batch[entity_table].seed_time
        x_dict = self.encoder(batch.tf_dict)

        rel_time_dict = self.temporal_encoder(
            seed_time, batch.time_dict, batch.batch_dict
        )

        for node_type, rel_time in rel_time_dict.items():
            x_dict[node_type] = x_dict[node_type] + rel_time

        for node_type, embedding in self.embedding_dict.items():
            x_dict[node_type] = x_dict[node_type] + embedding(batch[node_type].n_id)

        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
            batch.num_sampled_nodes_dict,
            batch.num_sampled_edges_dict,
        )

        return self.head(x_dict[entity_table][: seed_time.size(0)])

    def forward_dst_readout(
        self,
        batch: HeteroData,
        entity_table: NodeType,
        dst_table: NodeType,
    ) -> Tensor:
        if self.id_awareness_emb is None:
            raise RuntimeError(
                "id_awareness must be set True to use forward_dst_readout"
            )
        seed_time = batch[entity_table].seed_time
        x_dict = self.encoder(batch.tf_dict)
        # Add ID-awareness to the root node
        x_dict[entity_table][: seed_time.size(0)] += self.id_awareness_emb.weight

        rel_time_dict = self.temporal_encoder(
            seed_time, batch.time_dict, batch.batch_dict
        )

        for node_type, rel_time in rel_time_dict.items():
            x_dict[node_type] = x_dict[node_type] + rel_time

        for node_type, embedding in self.embedding_dict.items():
            x_dict[node_type] = x_dict[node_type] + embedding(batch[node_type].n_id)

        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
        )

        return self.head(x_dict[dst_table])


model = Model(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)


# if you try out different RelBench tasks you will need to change these
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
epochs = 10

In [77]:
def train() -> float:
    model.train()

    loss_accum = count_accum = 0
    for batch in tqdm(loader_dict["train"]):
        batch = batch.to(device)

        optimizer.zero_grad()
        pred = model(
            batch,
            seller_revenue_task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred

        loss = loss_fn(pred.float(), batch[entity_table].y.float())
        loss.backward()
        optimizer.step()

        loss_accum += loss.detach().item() * pred.size(0)
        count_accum += pred.size(0)

    return loss_accum / count_accum


@torch.no_grad()
def test(loader: NeighborLoader) -> np.ndarray:
    model.eval()

    pred_list = []
    for batch in loader:
        batch = batch.to(device)
        pred = model(
            batch,
            seller_revenue_task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
    return torch.cat(pred_list, dim=0).numpy()

In [87]:
state_dict = None
best_val_metric = -math.inf if higher_is_better else math.inf
for epoch in range(1, epochs + 1):
    train_loss = train()
    val_pred = test(loader_dict["val"])
    val_metrics = seller_revenue_task.evaluate(val_pred, val_table)
    print(f"Epoch: {epoch:02d}, Train loss: {train_loss}, Val metrics: {val_metrics}")

    if (higher_is_better and val_metrics[tune_metric] > best_val_metric) or (
        not higher_is_better and val_metrics[tune_metric] < best_val_metric
    ):
        best_val_metric = val_metrics[tune_metric]
        state_dict = copy.deepcopy(model.state_dict())


model.load_state_dict(state_dict)
val_pred = test(loader_dict["val"])
val_metrics = seller_revenue_task.evaluate(val_pred, val_table)
print(f"Best Val metrics: {val_metrics}")

test_pred = test(loader_dict["test"])
test_metrics = seller_revenue_task.evaluate(test_pred)
print(f"Best test metrics: {test_metrics}")

  0%|          | 0/21 [00:00<?, ?it/s]


ImportError: 'NeighborSampler' requires either 'pyg-lib' or 'torch-sparse'